In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from datetime import datetime
import nltk  
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
wordnet_lemmatizer = WordNetLemmatizer()
pd.reset_option('display.float_format')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
cd /Users/chiragrakholiya/Downloads/Project2/

/Users/chiragrakholiya/Downloads/Project2


In [3]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

data = pd.read_csv('hn2018_2019.csv', parse_dates=['Created At'], date_parser=dateparse)
data.head()

,Unnamed: 0,Object ID,Title,Post Type,Author,Created At,URL,Points,Number of Comments
0,2294635,16043786,Top Hacker News Books in 2017,story,0x54MUR41,2018-01-01 00:06:16,https://hackernewsbooks.com/year/2017,1,0.0
1,2294636,16043817,Beijing enjoys best winter air quality in five...,story,gpetukhov,2018-01-01 00:15:13,http://www.scmp.com/news/china/policies-politi...,2,0.0
2,2294637,16043825,Ask HN: Any domain name registrars that don't ...,ask_hn,glockenspielen,2018-01-01 00:16:54,NaN,1,2.0
3,2294638,16043845,Controversial Therapy Has Led to Death Threats...,story,cpncrunch,2018-01-01 00:23:05,https://www.buzzfeed.com/tomchivers/inside-the...,2,0.0
4,2294639,16043851,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,geraldbauer,2018-01-01 00:25:05,https://planetruby.github.io/advent2017/ruby3x3,2,0.0


In [5]:
type(data['Created At'][0])
split_date=pd.datetime(2019,1,1)
df_train = data.loc[data['Created At'] < split_date]
df_test = data.loc[data['Created At'] >= split_date]

In [6]:
Training_data=df_train["Title"]
Training_targets=df_train["Post Type"]
Testing_data=df_test["Title"]
Testing_targets=df_test["Post Type"]

In [7]:
def preprocess(sentence):
    sentence=sentence.encode("ascii", errors="ignore").decode()
    sentence = sentence.lower()
    #tokenizer = RegexpTokenizer(r'\w+')
    #tk = MWETokenizer([('ask', 'hn:'), ('show', 'hn:')])
    tokens = sentence.split()
    tokens=[i.strip('+-:?*_,$""~#%\'\|\!=') for i in tokens]
    lemma_words=[wordnet_lemmatizer.lemmatize(w) for w in tokens]
    Filtered_word = [s for s in lemma_words if (len(s) >= 2 and len(s)<=9)]           #Filtering the words len(word)>=2 and len(word)<=9
    return Filtered_word

In [8]:
count_vect = CountVectorizer(tokenizer=preprocess,stop_words=None)
X_train_counts = count_vect.fit_transform(Training_data)
X_train_counts

<276981x71051 sparse matrix of type '<class 'numpy.int64'>'
	with 1963814 stored elements in Compressed Sparse Row format>

In [9]:
len(count_vect.vocabulary_)


71051

In [10]:
df_train['tokenized_words'] = df_train['Title'].map(lambda s:preprocess(s))  #.apply(lambda row): nltk.tokenize.WhitespaceTokenizer().tokenize(row['Title']), axis=1)

/Users/chiragrakholiya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
Unique_posts=df_train['Post Type'].unique().tolist()

In [12]:
count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function preprocess at 0x1a25792050>,
                vocabulary=None)

In [13]:
rows = list()
for row in df_train[['Post Type', 'tokenized_words']].iterrows():
    r = row[1]
    for word in r.tokenized_words:
        rows.append((r['Post Type'], word)) #wordnet_lemmatizer.lemmatize(word, pos="v")))


In [14]:
dictionary = pd.DataFrame(rows, columns=['Post Type', 'word'])
dictionary = dictionary[dictionary['word'].isin(list(count_vect.vocabulary_.keys()))]
dictionary = dictionary[(dictionary.word.str.len() > 2 ) & (dictionary.word.str.len() <= 9)] 
dictionary.head(10)

,Post Type,word
0,story,top
1,story,hacker
2,story,news
3,story,book
5,story,2017
6,story,beijing
7,story,enjoys
8,story,best
9,story,winter
10,story,air


In [15]:
dictionary.sort_values(by=['word'])
#a['word'][3]

,Post Type,word
731513,story,&#43;
1193127,story,&amp;
1457115,story,&amp;
1733552,story,&amp;
1276861,story,&amp;
...,...,...
729741,story,{perfect}
5650,story,{poems}
1537942,story,{pytorch
250580,story,{wrong


In [16]:
Filtered_Vocab = dictionary['word'].unique().tolist()
print("The top 10 Unique words in our dictionary with their occurence are:\n{}".format(Filtered_Vocab[0:5]))
Filtered_Vocab.sort()
print("Total Unique words in our dictionary :",len(Filtered_Vocab))

The top 10 Unique words in our dictionary with their occurence are:
['top', 'hacker', 'news', 'book', '2017']
Total Unique words in our dictionary : 70161


In [17]:
Total_words_each_cat={}
for i in range(len(Unique_posts)):
    Total_words_each_cat[Unique_posts[i]]=len(dictionary.loc[dictionary['Post Type']== '{}'.format(Unique_posts[i])])

Total_words_each_cat

{'story': 1540249, 'ask_hn': 88663, 'show_hn': 68156, 'poll': 171}

In [18]:
Unique_words_each_cat={}
for i in range(len(Unique_posts)):
    Unique_words_each_cat[Unique_posts[i]]=len(dictionary.loc[dictionary['Post Type']=='{}'.format(Unique_posts[i])]['word'].unique())

Unique_words_each_cat

{'story': 66598, 'ask_hn': 7456, 'show_hn': 9578, 'poll': 94}

In [19]:
#def apperance_in_catergory_story(vocab):
#    Count_list=[]
#    for post in range(len(Unique_posts)):
#        try:
#            Count_list.append(dictionary.loc[dictionary['word']=='{}'.format(vocab)].groupby(['Post Type']).get_group('{}'.format(Unique_posts[post])).word.value_counts().to_frame().rename(columns={'word':'num_of_times'})['num_of_times'][0])
#        except KeyError:
#            Count_list.append(0)
#   
#    return Count_list

In [20]:
myfile = open('wordlength-model.txt', 'w',buffering=4096)
#myfile.write("Counter  Word  Freq_in_Story  Cond_prob_sty  Fq_in_ah  Cond_pr_ah  Fq_in_show_hn  Cond_prob_sh  Freq_in_poll  Cond_prob_poll\n")
for word in range(len(Filtered_Vocab)):
    Count_list=[]
    for post in range(len(Unique_posts)):
        try:
            Count_list.append(dictionary.loc[dictionary['word']=='{}'.format(Filtered_Vocab[word])].groupby(['Post Type']).get_group('{}'.format(Unique_posts[post])).word.value_counts().to_frame().rename(columns={'word':'num_of_times'})['num_of_times'][0])
        except KeyError:
            Count_list.append(0)
        
    myfile.write("{}  {}  {} {}  {}  {}  {}  {}  {}  {}\n".format(word+1,Filtered_Vocab[word],Count_list[0],format((Count_list[0]+0.5)/(Total_words_each_cat['story']+Unique_words_each_cat['story']),'.8f'),Count_list[1],format((Count_list[1]+0.5)/(Total_words_each_cat['ask_hn']+Unique_words_each_cat['ask_hn']),'.8f'),Count_list[2],format((Count_list[2]+0.5)/(Total_words_each_cat['show_hn']+Unique_words_each_cat['show_hn']),'.8f'),Count_list[3],format((Count_list[3]+0.5)/(Total_words_each_cat['poll']+Unique_words_each_cat['poll']),'.8f')))

myfile.close() 

KeyboardInterrupt: 

In [21]:
#tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
#X_train_tf = tf_transformer.transform(X_train_counts)
#X_train_tf

In [22]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(276981, 71051)

In [23]:
clf = MultinomialNB(alpha=0.5, fit_prior=True, class_prior=None).fit(X_train_tfidf,Training_targets)

In [25]:
Testing_counts = count_vect.transform(Testing_data)
Testing_tfidf = tfidf_transformer.transform(Testing_counts)

predicted = clf.predict_proba(Testing_tfidf)
predicted_string = clf.predict(Testing_tfidf)

In [26]:
myfile = open('wordlength-result.txt', 'w',buffering=4096)
for doc, pred_class, category , actual in zip(Testing_data.tolist(),predicted_string, predicted ,Testing_targets):
     myfile.write('%r  %s  %f  %f  %f  %f  %s %s' % (doc, pred_class ,category[3] ,category[0],category[2],category[1],actual,'right\n' if pred_class == actual else 'wrong\n'))
myfile.close()   

In [27]:
np.mean(predicted_string == Testing_targets)

0.9451590569544146